# LINEAR REGRESS PARAMS (YIELD TO PARAMS)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import scipy
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import statsmodels.formula.api as sm
import copy
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import sklearn
from math import sqrt
from functools import reduce
import numpy as np
#import ee
#import geemap
import geopandas as gpd
import rioxarray as rxr
import rasterstats as rs
from rasterio.plot import show
import rasterio

%matplotlib inline

# IRRIGATED PLOTS VS PARAMS

In [8]:
startYear = 2013
endYear = 2017

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JAS.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Irrigated Rice']
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'
ndvi = points.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# PCP
param_name = 'pcp'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JAS.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
pcp = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# LST DAY
param_name = 'LST_Day_1km'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JAS.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
lstday = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# LST NIGHT
param_name = 'LST_Night_1km'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JAS.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
lstnight = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# ET
param_name = 'et'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JAS.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
et = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# PET
param_name = 'pet'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JAS.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
pet = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# ELEVATION
param_name = 'elevation'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_ch3_elevation.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
elevation = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')
elevation['Date'] = endYear

# SLOPE
param_name = 'slope'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_ch3_slope.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
slope = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')
slope['Date'] = endYear

# Subset temporally and pivot to table for linear regression

merged = pd.concat([ndvi,pcp, lstday, lstnight, et, pet, elevation, slope])
subbed = merged[(merged.Date <= endYear) & (merged.Date >= startYear)].groupby(['ADM3_EN','param']).mean().reset_index(drop=False)
df = subbed.pivot(index="ADM3_EN", columns=["param"],values="value").reset_index(drop=False)

# Yield Data
actual = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/iloilo_rice_data_cleaned_imputed_new.csv')
actual = actual[(actual.param == 'yield') & (actual.ecosystem == 'irri')].reset_index(drop=True)
actual = actual[(actual.year >= startYear) & (actual.year <= endYear)].groupby('muni').mean().reset_index(drop=False)
actual = actual.rename(columns={'value':'RiceYield'})

# Merge Datasets
merged_df = pd.merge(actual, df, left_on='muni', right_on='ADM3_EN')
merged_df

# Apply the regression Equation
#formula = 'RiceYield ~ ndvi + pcp + LST_Day_1km + LST_Night_1km + et + pet + slope + elevation'
formula = 'RiceYield ~ ndvi + pcp + LST_Day_1km  +  pet + slope'
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged_df).fit()
print(results.summary())
#print(results.pvalues)


                            OLS Regression Results                            
Dep. Variable:              RiceYield   R-squared:                       0.575
Model:                            OLS   Adj. R-squared:                  0.397
Method:                 Least Squares   F-statistic:                     3.242
Date:                Fri, 07 Jan 2022   Prob (F-statistic):             0.0441
Time:                        01:00:29   Log-Likelihood:                 3.5502
No. Observations:                  18   AIC:                             4.900
Df Residuals:                      12   BIC:                             10.24
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -2.0985      4.067     -0.516      

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# RAINFED PLOTS VS PARAMS

In [81]:
startYear = 2013
endYear = 2017

# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JFM.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Rain-fed Rice']
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'
ndvi = points.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# PCP
param_name = 'pcp'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
pcp = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# LST DAY
param_name = 'LST_Day_1km'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
lstday = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# LST NIGHT
param_name = 'LST_Night_1km'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
lstnight = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# ET
param_name = 'et'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
et = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# PET
param_name = 'pet'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
pet = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# ELEVATION
param_name = 'elevation'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_ch3_elevation.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
elevation = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')
elevation['Date'] = 2017

# SLOPE
param_name = 'slope'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_ch3_slope.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
slope = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')
slope['Date'] = 2017

# Subset temporally and pivot to table for linear regression

merged = pd.concat([ndvi,pcp, lstday, lstnight, et, pet, elevation, slope])
subbed = merged[(merged.Date <= endYear) & (merged.Date >= startYear)].groupby(['ADM3_EN','param']).mean().reset_index(drop=False)
df = subbed.pivot(index="ADM3_EN", columns=["param"],values="value").reset_index(drop=False)
#Replace Duenas and City Of Passi
df = df.replace('Dueñas', 'Duenas')
df = df.replace('City of Passi', 'Passi City')


# Yield Data
actual = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/iloilo_rice_data_cleaned_imputed_new.csv')
actual = actual[(actual.param == 'yield') & (actual.ecosystem == 'rf')].reset_index(drop=True)
actual = actual[(actual.year >= startYear) & (actual.year <= endYear)].groupby('muni').mean().reset_index(drop=False)
actual = actual.rename(columns={'value':'RiceYield'})

# Merge Datasets
merged_df = pd.merge(actual, df, left_on='muni', right_on='ADM3_EN')
merged_df

merged_df = merged_df.loc[merged_df['ADM3_EN'].isin([
'Carles',
'Lambunao',
'Calinog',
'Duenas',
'Estancia',
'Sta Barbara',
'Maasin',
'Cabatuan',
'Leon',
'Bingawan',
'Passi City',
'Badiangan',
'San Dionisio',
'Balasan',
'Alimodian',
'Guimbal',
'New Lucena',
'Igbaras',
'Tubungan',
'Janiuay',
'San Enrique',
'Iloilo City',
'Batad' 
])]

# Apply the regression Equation
##formula = 'RiceYield ~ ndvi + pcp + LST_Day_1km + LST_Night_1km + et + pet + slope + elevation'
formula = 'RiceYield ~ slope '
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=merged_df).fit()
print(results.summary())
#print(results.pvalues)
#merged_df

                            OLS Regression Results                            
Dep. Variable:              RiceYield   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.033
Method:                 Least Squares   F-statistic:                    0.3897
Date:                Tue, 21 Dec 2021   Prob (F-statistic):              0.540
Time:                        02:12:54   Log-Likelihood:                -5.6498
No. Observations:                  20   AIC:                             15.30
Df Residuals:                      18   BIC:                             17.29
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.4444      0.126     27.401      0.0

In [37]:
merged_df

,muni,year,RiceYield,ADM3_EN,LST_Day_1km,LST_Night_1km,elevation,et,ndvi,pcp,pet,slope
1,Alimodian,2015,3.384596,Alimodian,30.641155,21.805687,161.947368,25.820919,2.059342,203.595577,51.463353,4.135536
3,Badiangan,2015,3.881851,Badiangan,29.364824,21.388511,71.357143,27.244087,2.258619,240.997659,49.363750,1.676291
4,Balasan,2015,3.087230,Balasan,29.728595,21.601695,21.777778,27.501977,1.996301,378.242723,50.960844,2.001327
9,Bingawan,2015,3.307013,Bingawan,28.938020,20.987163,87.166667,26.585278,2.266907,287.182462,49.674815,1.189152
10,Cabatuan,2015,3.463998,Cabatuan,31.310868,21.900910,68.000000,22.829387,1.969970,226.979032,48.452315,1.439763
11,Calinog,2015,3.769642,Calinog,29.825592,20.979830,89.050000,25.959236,2.189283,258.823963,48.598194,1.965074
12,Carles,2015,2.489368,Carles,29.576790,22.346869,16.647059,28.775963,1.932326,331.980659,51.517778,2.276907
15,Duenas,2015,3.174676,Duenas,30.516409,21.244963,48.400000,21.850377,2.138055,239.735258,46.457686,1.207737
17,Estancia,2015,3.504090,Estancia,29.493126,21.871567,21.600000,27.206000,2.065696,346.760118,50.357028,1.956394
18,Guimbal,2015,3.268034,Guimbal,32.612012,23.078283,42.222222,23.221790,1.897802,164.306025,52.200679,1.584619
